In [1]:
import pandas as pd
import numpy as np
import os

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.metrics import f1_score, confusion_matrix, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold
from catboost import CatBoostClassifier
from itertools import product
from sklearn import svm

In [2]:
os.chdir('C:\\Users\\fleur\\2021 P-SAT\\주제분석\\주제분석 2주차\\데이터')

In [3]:
train_dog = pd.read_csv('final_train_dog.csv', engine='python')
train_cat = pd.read_csv('flt_train_cat.csv', engine='python')

In [58]:
test_dog = pd.read_csv('final_test_dog.csv', engine = 'python')
test_cat = pd.read_csv('flt_test_cat.csv', engine = 'python')

In [4]:
dog_cate = ['neuterYN', 'sex', 'group_akc', 'color']
dog_nume = ['weight_kg', 'positives', 'negatives', 'grdp', 'economy', 'hospital_num']
dog_target = ['adoptionYN']

cat_cate = ['season', 'neuterYN', 'sex', 'hair', 'color']
cat_nume = ['age', 'disease_cnt', 'positives', 'negatives', 'grdp', 'economy', 'hospital_num']
cat_target = ['adoptionYN']

In [5]:
train_dog[dog_cate] = train_dog[dog_cate].astype('category')
train_cat[cat_cate] = train_cat[cat_cate].astype('category')

In [59]:
test_dog[dog_cate] = test_dog[dog_cate].astype('category')
test_cat[cat_cate] = test_cat[cat_cate].astype('category')

In [6]:
dog_trainy = train_dog['adoptionYN']
dog_trainx = train_dog.drop('adoptionYN', axis = 1)

In [57]:
cat_trainy = train_cat['adoptionYN']
cat_trainx = train_cat.drop('adoptionYN', axis = 1)

In [60]:
dog_testy = test_dog['adoptionYN']
dog_testx = test_dog.drop('adoptionYN', axis = 1)

cat_testy = test_cat['adoptionYN']
cat_testx = test_cat.drop('adoptionYN', axis = 1)

## 망망이 튜닝

In [35]:
def expand_grid(dictionary):
   return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

dictionary = {'C' : [0.1, 0.5, 1.0, 5, 10, 100],
              'kernel' : ['linear', 'rbf', 'sigmoid'],
              'gamma' : [0.1, 0.5, 1, 10, 100]}

dog_tuned_svm = expand_grid(dictionary)
dog_tuned_svm['F1'] = None
dog_tuned_svm['Accuracy'] = None
dog_tuned_svm

,C,kernel,gamma,F1,Accuracy
0,0.1,linear,0.1,None,None
1,0.1,linear,0.5,None,None
2,0.1,linear,1.0,None,None
3,0.1,linear,10.0,None,None
4,0.1,linear,100.0,None,None
...,...,...,...,...,...
85,100.0,sigmoid,0.1,None,None
86,100.0,sigmoid,0.5,None,None
87,100.0,sigmoid,1.0,None,None
88,100.0,sigmoid,10.0,None,None


In [36]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 613)

f1 = []
accuracy = []

for i in range(0,89):
    for train_index, test_index in kfold.split(dog_trainx, dog_trainy):
        print("TRAIN:", train_index, "TEST:", test_index)
    
        X_train, X_test = dog_trainx.iloc[train_index], dog_trainx.iloc[test_index]
        y_train, y_test = dog_trainy.iloc[train_index], dog_trainy.iloc[test_index]
        
        # catboost encoding
        cbe_encoder = CatBoostEncoder()
        cbe_encoder.fit(X_train, y_train)
        train_cbe = cbe_encoder.transform(X_train)
        val_cbe = cbe_encoder.transform(X_test)
        
        # standardization
        sc = StandardScaler()
        sc.fit(train_cbe[dog_nume])
        train_cbe[dog_nume] = sc.transform(train_cbe[dog_nume])
        val_cbe[dog_nume] = sc.transform(val_cbe[dog_nume])
        
        svc = svm.SVC(C = dog_tuned_svm.iloc[i,0],
                      kernel = dog_tuned_svm.iloc[i,1],
                      gamma = dog_tuned_svm.iloc[i,2],
                      random_state = 613)
        
        svc.fit(train_cbe, y_train)
        y_pred = svc.predict(val_cbe)
        
        # F1 score
        f1.append(f1_score(y_test, y_pred))
        # accuracy
        accuracy.append(accuracy_score(y_test, y_pred))

    dog_tuned_svm.iloc[i,3] = np.mean(f1)
    dog_tuned_svm.iloc[i,4] = np.mean(accuracy)
    f1 = []
    accuracy = []

TRAIN: [    0     2     3 ... 55918 55919 55920] TEST: [    1     4     7 ... 55917 55921 55922]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   10    14    18 ... 55911 55912 55914]
TRAIN: [    1     4     5 ... 55918 55921 55922] TEST: [    0     2     3 ... 55910 55919 55920]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   12    15    25 ... 55900 55913 55918]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [    5     8    11 ... 55901 55904 55907]
TRAIN: [    0     2     3 ... 55918 55919 55920] TEST: [    1     4     7 ... 55917 55921 55922]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   10    14    18 ... 55911 55912 55914]
TRAIN: [    1     4     5 ... 55918 55921 55922] TEST: [    0     2     3 ... 55910 55919 55920]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   12    15    25 ... 55900 55913 55918]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [    5     8    11 ... 55901 55904 55907]
TRAIN: [    0     2     3 ... 

TRAIN: [    0     2     3 ... 55918 55919 55920] TEST: [    1     4     7 ... 55917 55921 55922]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   10    14    18 ... 55911 55912 55914]
TRAIN: [    1     4     5 ... 55918 55921 55922] TEST: [    0     2     3 ... 55910 55919 55920]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   12    15    25 ... 55900 55913 55918]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [    5     8    11 ... 55901 55904 55907]
TRAIN: [    0     2     3 ... 55918 55919 55920] TEST: [    1     4     7 ... 55917 55921 55922]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   10    14    18 ... 55911 55912 55914]
TRAIN: [    1     4     5 ... 55918 55921 55922] TEST: [    0     2     3 ... 55910 55919 55920]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   12    15    25 ... 55900 55913 55918]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [    5     8    11 ... 55901 55904 55907]
TRAIN: [    0     2     3 ... 

TRAIN: [    0     2     3 ... 55918 55919 55920] TEST: [    1     4     7 ... 55917 55921 55922]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   10    14    18 ... 55911 55912 55914]
TRAIN: [    1     4     5 ... 55918 55921 55922] TEST: [    0     2     3 ... 55910 55919 55920]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   12    15    25 ... 55900 55913 55918]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [    5     8    11 ... 55901 55904 55907]
TRAIN: [    0     2     3 ... 55918 55919 55920] TEST: [    1     4     7 ... 55917 55921 55922]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   10    14    18 ... 55911 55912 55914]
TRAIN: [    1     4     5 ... 55918 55921 55922] TEST: [    0     2     3 ... 55910 55919 55920]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   12    15    25 ... 55900 55913 55918]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [    5     8    11 ... 55901 55904 55907]
TRAIN: [    0     2     3 ... 

TRAIN: [    0     2     3 ... 55918 55919 55920] TEST: [    1     4     7 ... 55917 55921 55922]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   10    14    18 ... 55911 55912 55914]
TRAIN: [    1     4     5 ... 55918 55921 55922] TEST: [    0     2     3 ... 55910 55919 55920]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   12    15    25 ... 55900 55913 55918]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [    5     8    11 ... 55901 55904 55907]
TRAIN: [    0     2     3 ... 55918 55919 55920] TEST: [    1     4     7 ... 55917 55921 55922]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   10    14    18 ... 55911 55912 55914]
TRAIN: [    1     4     5 ... 55918 55921 55922] TEST: [    0     2     3 ... 55910 55919 55920]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [   12    15    25 ... 55900 55913 55918]
TRAIN: [    0     1     2 ... 55920 55921 55922] TEST: [    5     8    11 ... 55901 55904 55907]
TRAIN: [    0     2     3 ... 

KeyboardInterrupt: 

In [50]:
# 9시 7분
dog_tuned_svm.iloc[0:62,].tail(30)

,C,kernel,gamma,F1,Accuracy
33,1.0,linear,10.0,0.410247,0.695921
34,1.0,linear,100.0,0.410247,0.695921
35,1.0,rbf,0.1,0.479814,0.711586
36,1.0,rbf,0.5,0.536783,0.726195
37,1.0,rbf,1.0,0.548024,0.730129
38,1.0,rbf,10.0,0.587178,0.739767
39,1.0,rbf,100.0,0.525814,0.716646
40,1.0,sigmoid,0.1,0.386205,0.553511
41,1.0,sigmoid,0.5,0.337928,0.514242
42,1.0,sigmoid,1.0,0.335636,0.51376


In [52]:
dog_tuned_svm.iloc[0:62,][dog_tuned_svm.iloc[0:62,3] == max(dog_tuned_svm.iloc[0:62,3])]

,C,kernel,gamma,F1,Accuracy
53,5.0,rbf,10.0,0.59132,0.736101


## 애옹이 튜닝

In [113]:
def expand_grid(dictionary):
   return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

dictionary = {'C' : [1, 5, 7, 10],
              'kernel' : ['linear', 'rbf', 'sigmoid'],
              'gamma' : [0.5, 1, 10, 15]}

cat_tuned_svm = expand_grid(dictionary)
cat_tuned_svm['F1'] = None
cat_tuned_svm['Accuracy'] = None
cat_tuned_svm

,C,kernel,gamma,F1,Accuracy
0,1,linear,0.5,None,None
1,1,linear,1.0,None,None
2,1,linear,10.0,None,None
3,1,linear,15.0,None,None
4,1,rbf,0.5,None,None
5,1,rbf,1.0,None,None
6,1,rbf,10.0,None,None
7,1,rbf,15.0,None,None
8,1,sigmoid,0.5,None,None
9,1,sigmoid,1.0,None,None


In [114]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 613)

f1 = []
accuracy = []

cat_trainx_dum = pd.concat([pd.get_dummies(cat_trainx[cat_cate]), cat_trainx[cat_nume]], axis = 1)

for i in range(0,48):
    for train_index, test_index in kfold.split(cat_trainx_dum, cat_trainy):
        print(i)
    
        X_train, X_test = cat_trainx_dum.iloc[train_index], cat_trainx_dum.iloc[test_index]
        y_train, y_test = cat_trainy.iloc[train_index], cat_trainy.iloc[test_index]
        
        # standardization
        sc = StandardScaler()
        sc.fit(X_train[cat_nume])
        X_train[cat_nume] = sc.transform(X_train[cat_nume])
        X_test[cat_nume] = sc.transform(X_test[cat_nume])
        
        svc = svm.SVC(C = cat_tuned_svm.iloc[i,0],
                      kernel = cat_tuned_svm.iloc[i,1],
                      gamma = cat_tuned_svm.iloc[i,2],
                      random_state = 613)
        
        svc.fit(X_train, y_train)
        y_pred = svc.predict(X_test)
        
        # F1 score
        f1.append(f1_score(y_test, y_pred))
        # accuracy
        accuracy.append(accuracy_score(y_test, y_pred))

    cat_tuned_svm.iloc[i,3] = np.mean(f1)
    cat_tuned_svm.iloc[i,4] = np.mean(accuracy)
    f1 = []
    accuracy = []

0


<ipython-input-114-827aa473af24>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[cat_nume] = sc.transform(X_train[cat_nume])
C:\Users\fleur\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
<ipython-input-114-827aa473af24>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

0


<ipython-input-114-827aa473af24>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[cat_nume] = sc.transform(X_train[cat_nume])
C:\Users\fleur\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
<ipython-input-114-827aa473af24>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

0


...

6


<ipython-input-114-827aa473af24>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[cat_nume] = sc.transform(X_train[cat_nume])
C:\Users\fleur\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
<ipython-input-114-827aa473af24>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

7


<ipython-input-114-827aa473af24>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[cat_nume] = sc.transform(X_train[cat_nume])
C:\Users\fleur\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
<ipython-input-114-827aa473af24>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

KeyboardInterrupt: 

In [63]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 613)

f1 = []
accuracy = []

for i in range(0,18):
    for train_index, test_index in kfold.split(cat_trainx, cat_trainy):
        print(i)
    
        X_train, X_test = cat_trainx.iloc[train_index], cat_trainx.iloc[test_index]
        y_train, y_test = cat_trainy.iloc[train_index], cat_trainy.iloc[test_index]
        
        # catboost encoding
        cbe_encoder = CatBoostEncoder()
        cbe_encoder.fit(X_train, y_train)
        train_cbe = cbe_encoder.transform(X_train)
        val_cbe = cbe_encoder.transform(X_test)
        
        # standardization
        sc = StandardScaler()
        sc.fit(train_cbe[cat_nume])
        train_cbe[cat_nume] = sc.transform(train_cbe[cat_nume])
        val_cbe[cat_nume] = sc.transform(val_cbe[cat_nume])
        
        svc = svm.SVC(C = cat_tuned_svm.iloc[i,0],
                      kernel = cat_tuned_svm.iloc[i,1],
                      gamma = cat_tuned_svm.iloc[i,2],
                      random_state = 613)
        
        svc.fit(train_cbe, y_train)
        y_pred = svc.predict(val_cbe)
        
        # F1 score
        f1.append(f1_score(y_test, y_pred))
        # accuracy
        accuracy.append(accuracy_score(y_test, y_pred))

    cat_tuned_svm.iloc[i,3] = np.mean(f1)
    cat_tuned_svm.iloc[i,4] = np.mean(accuracy)
    f1 = []
    accuracy = []

0
0
0
0
0
1
1
1
1
1
2
2
2
2
2
3
3
3
3
3
4
4
4
4
4
5
5
5
5
5
6
6
6
6
6
7
7
7
7
7
8
8
8
8
8
9
9
9
9
9
10
10
10
10
10
11
11
11
11
11
12
12
12
12
12
13
13
13
13
13
14
14
14
14
14
15
15
15
15
15
16
16
16
16
16
17
17
17
17
17


In [64]:
cat_tuned_svm[cat_tuned_svm['F1'] == max(cat_tuned_svm['F1'])]

,C,kernel,gamma,F1,Accuracy
13,5.0,rbf,1.0,0.583438,0.719362


## 망망이 평가

In [104]:
dog_trainx_dum = pd.concat([pd.get_dummies(dog_trainx[dog_cate]), dog_trainx[dog_nume]], axis = 1)

In [105]:
dog_testx_dum = pd.concat([pd.get_dummies(dog_testx[dog_cate]), dog_testx[dog_nume]], axis = 1)

In [106]:
# standardization
sc.fit(dog_trainx_dum[dog_nume])
dog_trainx_dum[dog_nume] = sc.transform(dog_trainx_dum[dog_nume])
dog_testx_dum[dog_nume] = sc.transform(dog_testx_dum[dog_nume])

In [128]:
# catboost encoding
cbe_encoder.fit(dog_trainx, dog_trainy)
dog_train_cbe_svm = cbe_encoder.transform(dog_trainx)
dog_test_cbe_svm = cbe_encoder.transform(dog_testx)

# standardization
sc.fit(dog_train_cbe_svm[dog_nume])
dog_train_cbe_svm[dog_nume] = sc.transform(dog_train_cbe_svm[dog_nume])
dog_test_cbe_svm[dog_nume] = sc.transform(dog_test_cbe_svm[dog_nume])

In [107]:
svc = svm.SVC(C = 5, kernel = 'rbf', gamma = 10, random_state = 613)
        
svc.fit(dog_trainx_dum, dog_trainy)
dog_ypred_svmd = svc.predict(dog_testx_dum)
        
# F1 score
dog_f1_svmd = f1_score(dog_testy, dog_ypred_svmd)
# accuracy
dog_acc_svmd = accuracy_score(dog_testy, dog_ypred_svmd)

In [108]:
dog_f1_svmd

0.4530931100007038

In [109]:
dog_acc_svmd

0.6757760347129506

In [129]:
svc = svm.SVC(C = 5, kernel = 'rbf', gamma = 10, random_state = 613)
        
svc.fit(dog_train_cbe_svm, dog_trainy)
dog_ypred_svm = svc.predict(dog_test_cbe_svm)
        
# F1 score
dog_f1_svm = f1_score(dog_testy, dog_ypred_svm)
# accuracy
dog_acc_svm = accuracy_score(dog_testy, dog_ypred_svm)

In [130]:
dog_f1_svm

0.5886204413016435

In [131]:
dog_acc_svm

0.7378588117489987

In [132]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(svc, random_state = 613).fit(dog_test_cbe_svm, dog_testy)
eli5.show_weights(perm, feature_names = dog_test_cbe_svm.columns.tolist())

Weight,Feature
0.0963 ± 0.0022,economy
0.0879 ± 0.0042,hospital_num
0.0865 ± 0.0040,grdp
0.0666 ± 0.0020,group_akc
0.0410 ± 0.0042,weight_kg
0.0328 ± 0.0028,positives
0.0175 ± 0.0021,negatives
0.0022 ± 0.0007,neuterYN
0.0008 ± 0.0010,color
0.0002 ± 0.0004,sex


## 애옹이 평가

In [133]:
# catboost encoding
cbe_encoder.fit(cat_trainx, cat_trainy)
cat_train_cbe_svm = cbe_encoder.transform(cat_trainx)
cat_test_cbe_svm = cbe_encoder.transform(cat_testx)
        
# standardization
sc.fit(cat_train_cbe_svm[cat_nume])
cat_train_cbe_svm[cat_nume] = sc.transform(cat_train_cbe_svm[cat_nume])
cat_test_cbe_svm[cat_nume] = sc.transform(cat_test_cbe_svm[cat_nume])
        
svc = svm.SVC(C = 5, kernel = 'rbf', gamma = 1, random_state = 613)
        
svc.fit(cat_train_cbe_svm, cat_trainy)
cat_ypred_svm = svc.predict(cat_test_cbe_svm)
        
# F1 score
cat_f1_svm = f1_score(cat_testy, cat_ypred_svm)
# accuracy
cat_acc_svm = accuracy_score(cat_testy, cat_ypred_svm)

In [134]:
cat_f1_svm

0.5860240963855422

In [135]:
cat_acc_svm

0.720013037809648

In [136]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(svc, random_state = 613).fit(cat_test_cbe_svm, cat_testy)
eli5.show_weights(perm, feature_names = cat_test_cbe_svm.columns.tolist())

Weight,Feature
0.0712 ± 0.0036,economy
0.0543 ± 0.0049,grdp
0.0531 ± 0.0036,negatives
0.0479 ± 0.0062,hospital_num
0.0443 ± 0.0047,positives
0.0388 ± 0.0046,disease_cnt
0.0166 ± 0.0027,age
0.0054 ± 0.0026,season
0.0042 ± 0.0015,hair
0.0033 ± 0.0016,sex


# catboost

In [84]:
def expand_grid(dictionary):
   return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

dictionary = {'iterations':[10,30,50,100,150,200],
              'learning_rate': [0.03, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5],
              'depth': [4, 6, 10]}

dog_paramtuned = expand_grid(dictionary)
dog_paramtuned['F1'] = None
dog_paramtuned['Accuracy'] = None
dog_paramtuned

,iterations,learning_rate,depth,F1,Accuracy
0,10,0.03,4,None,None
1,10,0.03,6,None,None
2,10,0.03,10,None,None
3,10,0.05,4,None,None
4,10,0.05,6,None,None
...,...,...,...,...,...
121,200,0.30,6,None,None
122,200,0.30,10,None,None
123,200,0.50,4,None,None
124,200,0.50,6,None,None


In [85]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 613)

f1 = []
accuracy = []

for i in range(0,126):
    for train_index, test_index in kfold.split(dog_trainx, dog_trainy):
        print(i)
    
        X_train, X_test = dog_trainx.iloc[train_index], dog_trainx.iloc[test_index]
        y_train, y_test = dog_trainy.iloc[train_index], dog_trainy.iloc[test_index]
        
        # catboost encoding
        cbe_encoder = CatBoostEncoder()
        cbe_encoder.fit(X_train, y_train)
        train_cbe = cbe_encoder.transform(X_train)
        val_cbe = cbe_encoder.transform(X_test)
        
        # standardization
        sc = StandardScaler()
        sc.fit(train_cbe[dog_nume])
        train_cbe[dog_nume] = sc.transform(train_cbe[dog_nume])
        val_cbe[dog_nume] = sc.transform(val_cbe[dog_nume])
        
        cb = CatBoostClassifier(iterations = dog_paramtuned.iloc[i,0], learning_rate = dog_paramtuned.iloc[i,1], 
                                depth = dog_paramtuned.iloc[i,2], loss_function = 'Logloss', random_seed = 613)
        
        cb.fit(train_cbe, y_train, early_stopping_rounds = 50, verbose=0)
        y_pred = cb.predict(val_cbe)
        
        # F1 score
        f1.append(f1_score(y_test, y_pred))
        # accuracy
        accuracy.append(accuracy_score(y_test, y_pred))

    dog_paramtuned.iloc[i,3] = np.mean(f1)
    dog_paramtuned.iloc[i,4] = np.mean(accuracy)
    f1 = []
    accuracy = []

0


CatBoostError: Invalid loss_function='F1': for classifier use Logloss, CrossEntropy, MultiClass, MultiClassOneVsAll or custom objective object

In [15]:
dog_paramtuned

,iterations,learning_rate,depth,F1,Accuracy
0,10,0.03,4,0.411677,0.696261
1,10,0.03,6,0.458056,0.703753
2,10,0.03,10,0.505969,0.715001
3,10,0.05,4,0.415192,0.696493
4,10,0.05,6,0.462548,0.705774
...,...,...,...,...,...
121,200,0.30,6,0.606891,0.743111
122,200,0.30,10,0.594332,0.729825
123,200,0.50,4,0.606158,0.744041
124,200,0.50,6,0.604731,0.739249


In [20]:
dog_paramtuned[dog_paramtuned['F1'] == max(dog_paramtuned['F1'])]

,iterations,learning_rate,depth,F1,Accuracy
118,200,0.2,6,0.607694,0.747188


In [21]:
dog_paramtuned[dog_paramtuned['Accuracy'] == max(dog_paramtuned['Accuracy'])]

,iterations,learning_rate,depth,F1,Accuracy
118,200,0.2,6,0.607694,0.747188


## 애옹이

In [78]:
def expand_grid(dictionary):
   return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

dictionary = {'iterations':[50,100,150,200,250,300],
              'learning_rate': [0.03, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5],
              'depth': [4, 6, 10]}

cat_tuned_cat = expand_grid(dictionary)
cat_tuned_cat['F1'] = None
cat_tuned_cat['Accuracy'] = None
cat_tuned_cat

,iterations,learning_rate,depth,F1,Accuracy
0,50,0.03,4,None,None
1,50,0.03,6,None,None
2,50,0.03,10,None,None
3,50,0.05,4,None,None
4,50,0.05,6,None,None
...,...,...,...,...,...
121,300,0.30,6,None,None
122,300,0.30,10,None,None
123,300,0.50,4,None,None
124,300,0.50,6,None,None


In [82]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 613)

f1 = []
accuracy = []

for i in range(0,126):
    for train_index, test_index in kfold.split(cat_trainx, cat_trainy):
        print(i)
    
        X_train, X_test = cat_trainx.iloc[train_index], cat_trainx.iloc[test_index]
        y_train, y_test = cat_trainy.iloc[train_index], cat_trainy.iloc[test_index]
        
        # catboost encoding
        cbe_encoder = CatBoostEncoder()
        cbe_encoder.fit(X_train, y_train)
        train_cbe = cbe_encoder.transform(X_train)
        val_cbe = cbe_encoder.transform(X_test)
        
        # standardization
        sc = StandardScaler()
        sc.fit(train_cbe[cat_nume])
        train_cbe[cat_nume] = sc.transform(train_cbe[cat_nume])
        val_cbe[cat_nume] = sc.transform(val_cbe[cat_nume])
        
        cb = CatBoostClassifier(iterations = cat_tuned_cat.iloc[i,0], learning_rate = cat_tuned_cat.iloc[i,1], 
                                depth = cat_tuned_cat.iloc[i,2], loss_function = 'Logloss', random_seed = 613)
        
        cb.fit(train_cbe, y_train, early_stopping_rounds = 50, verbose=0)
        y_pred = cb.predict(val_cbe)
        
        # F1 score
        f1.append(f1_score(y_test, y_pred))
        # accuracy
        accuracy.append(accuracy_score(y_test, y_pred))

    cat_tuned_cat.iloc[i,3] = np.mean(f1)
    cat_tuned_cat.iloc[i,4] = np.mean(accuracy)
    f1 = []
    accuracy = []

0
0
0
0
0
1
1
1
1
1
2
2
2
2
2
3
3
3
3
3
4
4
4
4
4
5
5
5
5
5
6
6
6
6
6
7
7
7
7
7
8
8
8
8
8
9
9
9
9
9
10
10
10
10
10
11
11
11
11
11
12
12
12
12
12
13
13
13
13
13
14
14
14
14
14
15
15
15
15
15
16
16
16
16
16
17
17
17
17
17
18
18
18
18
18
19
19
19
19
19
20
20
20
20
20
21
21
21
21
21
22
22
22
22
22
23
23
23
23
23
24
24
24
24
24
25
25
25
25
25
26
26
26
26
26
27
27
27
27
27
28
28
28
28
28
29
29
29
29
29
30
30
30
30
30
31
31
31
31
31
32
32
32
32
32
33
33
33
33
33
34
34
34
34
34
35
35
35
35
35
36
36
36
36
36
37
37
37
37
37
38
38
38
38
38
39
39
39
39
39
40
40
40
40
40
41
41
41
41
41
42
42
42
42
42
43
43
43
43
43
44
44
44
44
44
45
45
45
45
45
46
46
46
46
46
47
47
47
47
47
48
48
48
48
48
49
49
49
49
49
50
50
50
50
50
51
51
51
51
51
52
52
52
52
52
53
53
53
53
53
54
54
54
54
54
55
55
55
55
55
56
56
56
56
56
57
57
57
57
57
58
58
58
58
58
59
59
59
59
59
60
60
60
60
60
61
61
61
61
61
62
62
62
62
62
63
63
63
63
63
64
64
64
64
64
65
65
65
65
65
66
66
66
66
66
67
67
67
67
67
68
68
68
68
68
69
69
69
69
69


In [83]:
cat_tuned_cat[cat_tuned_cat['F1'] == max(cat_tuned_cat['F1'])]

,iterations,learning_rate,depth,F1,Accuracy
56,150,0.2,10,0.632802,0.744739


## 망망이 평가

In [120]:
# catboost encoding
cbe_encoder.fit(dog_trainx, dog_trainy)
dog_train_cbe_cat = cbe_encoder.transform(dog_trainx)
dog_test_cbe_cat = cbe_encoder.transform(dog_testx)
        
# standardization
sc.fit(dog_train_cbe_cat[dog_nume])
dog_train_cbe_cat[dog_nume] = sc.transform(dog_train_cbe_cat[dog_nume])
dog_test_cbe_cat[dog_nume] = sc.transform(dog_test_cbe_cat[dog_nume])
        
cb = CatBoostClassifier(iterations = 200, learning_rate = 0.2, depth = 6, loss_function = 'Logloss', random_seed = 613)
        
cb.fit(dog_train_cbe_cat, dog_trainy, eval_set = (dog_test_cbe_cat, dog_testy), early_stopping_rounds = 50, verbose=0)
dog_ypred_cat = cb.predict(dog_test_cbe_cat)
        
# F1 score
dog_f1_cat = f1_score(dog_testy, dog_ypred_cat)
# accuracy
dog_acc_cat = accuracy_score(dog_testy, dog_ypred_cat)

In [121]:
dog_f1_cat

0.6072517350976195

In [122]:
dog_acc_cat

0.7473714953271028

In [123]:
### 변수 중요도 시각화
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(cb, random_state = 613).fit(dog_test_cbe_cat, dog_testy)
eli5.show_weights(perm, feature_names = dog_test_cbe_cat.columns.tolist())

Weight,Feature
0.0721 ± 0.0019,group_akc
0.0593 ± 0.0028,hospital_num
0.0468 ± 0.0052,grdp
0.0378 ± 0.0041,economy
0.0289 ± 0.0013,weight_kg
0.0182 ± 0.0015,positives
0.0041 ± 0.0010,negatives
0.0034 ± 0.0022,color
0.0028 ± 0.0008,neuterYN
0.0007 ± 0.0005,sex


## 애옹이 평가

In [124]:
# catboost encoding
cbe_encoder.fit(cat_trainx, cat_trainy)
cat_train_cbe_cat = cbe_encoder.transform(cat_trainx)
cat_test_cbe_cat = cbe_encoder.transform(cat_testx)
        
# standardization
sc.fit(cat_train_cbe_cat[cat_nume])
cat_train_cbe_cat[cat_nume] = sc.transform(cat_train_cbe_cat[cat_nume])
cat_test_cbe_cat[cat_nume] = sc.transform(cat_test_cbe_cat[cat_nume])
        
cb = CatBoostClassifier(iterations = 150, learning_rate = 0.2, depth = 10, loss_function = 'Logloss', random_seed = 613)
        
cb.fit(cat_train_cbe_cat, cat_trainy, eval_set = (cat_test_cbe_cat, cat_testy), early_stopping_rounds = 50, verbose=0)
cat_ypred_cat = cb.predict(cat_test_cbe_cat)
        
# F1 score
cat_f1_cat = f1_score(cat_testy, cat_ypred_cat)
# accuracy
cat_acc_cat = accuracy_score(cat_testy, cat_ypred_cat)

In [125]:
cat_f1_cat

0.6363349131121643

In [126]:
cat_acc_cat

0.7498913515862669

In [127]:
### 변수 중요도 시각화
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(cb, random_state = 613).fit(cat_test_cbe_cat, cat_testy)
eli5.show_weights(perm, feature_names = cat_test_cbe_cat.columns.tolist())

Weight,Feature
0.0457 ± 0.0043,grdp
0.0426 ± 0.0027,negatives
0.0320 ± 0.0066,hospital_num
0.0287 ± 0.0038,disease_cnt
0.0262 ± 0.0040,season
0.0248 ± 0.0031,economy
0.0238 ± 0.0030,positives
0.0202 ± 0.0063,sex
0.0115 ± 0.0010,hair
0.0078 ± 0.0023,age
